In [80]:
from robobrowser import RoboBrowser
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

def get_number(s):
    try:
        return float(s.replace(",",""))
    except ValueError:
        return None

#Open URL and create RoboBrowser Instance
browser = RoboBrowser(parser='html.parser', history=False)
dates = pd.date_range('2016-01-02', periods=3, freq='2D').format('YYYY-MM-dd')[1:] # url parameter is last date shown on website, website shows two dates for one request
print(dates)
results = pd.DataFrame()

for d in dates:
    url='https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/' + d + '/DE'
    print(url)
    browser.open(url)  
    soup = bs(str(browser.parsed), 'html.parser') # get html
    dayData = soup.find("table") # select table
    #print(soup.prettify())
    myDates = pd.date_range(pd.to_datetime(d) - pd.to_timedelta(1, 'D'), periods=2, freq="D") # create two shown dates
    tableHeader = dayData.find("tr", class_=None) # get table header row
    columnNames = [p.get_text() for p in tableHeader.findAll("th")][2:11]
    priceRows = dayData.findAll("tr", class_=None)[1:] # get rows with €/Mwh with hourly interval
    hours = [] # get table row names
    for r in priceRows[:24]:
        hours.append(r.find("td", class_="title").get_text().strip()) 
    # get prices for both days on website
    prices = np.zeros((2, 24, 9))
    for i in range(24):
        prices[0][i] = [get_number(p.get_text()) for p in priceRows[i].findAll("td")][3:12]
        prices[1][i] = [get_number(p.get_text()) for p in priceRows[i].findAll("td")][13:]
    #print(prices)
    for i in range(2):
        df = pd.DataFrame(prices[i])
        df.columns = columnNames
        df["hours"] = hours
        df = df.set_index("hours", drop=False)
        df["date"] = myDates[i]
        if results.empty:
            results = df
        else:
            results = results.append(df)
        #print(df)
results = results.set_index(["date", "hours"])
print(results)



['2016-01-01', '2016-01-03', '2016-01-05']
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-01-01/DE
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-01-03/DE
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-01-05/DE
                    Low(€/MWh)  High(€/MWh)  Last(€/MWh)  Weight. Avg.(€/MWh)  \
date       hours                                                                
2015-12-31 00 - 01       13.14        31.99        19.90                16.06   
           01 - 02        8.00        35.00        35.00                17.15   
           02 - 03        3.00        30.00        27.00                12.96   
           03 - 04        0.10        25.00        14.00                 8.60   
           04 - 05       -5.41        15.00         0.50                 3.27   
           05 - 06       -2.96        27.00         0.00                 3.56   
           06 - 07       -4.00        25.00  